## format vcf to runPM input (chr22; afreq cutoff 0.01)

### Get variants with afreq > 0.01 from annotation (AF)

In [ ]:
anno = pd.read_csv('../MWE/annotation/EFIGA_NIALOAD_chr22.hg38.hg38_multianno.csv')

In [ ]:
anno

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,CLNDISDB,CLNREVSTAT,CLNSIG,DN ID,Patient ID,Phenotype,Platform,Study,Pubmed ID,Otherinfo1
0,22,10510356,10510356,T,A,intergenic,NONE;FRG1FP,dist=NONE;dist=430241,.,.,...,.,.,.,.,.,.,.,.,.,chr22:10510356:T:A
1,22,10510356,10510357,TA,T,intergenic,NONE;FRG1FP,dist=NONE;dist=430240,.,.,...,.,.,.,.,.,.,.,.,.,chr22:10510356:TA:T
2,22,10510356,10510357,TA,*,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,chr22:10510356:TA:*
3,22,10511359,10511359,T,G,intergenic,NONE;FRG1FP,dist=NONE;dist=429238,.,.,...,.,.,.,.,.,.,.,.,.,chr22:10511359:T:G
4,22,10511359,10511361,TAA,T,intergenic,NONE;FRG1FP,dist=NONE;dist=429236,.,.,...,.,.,.,.,.,.,.,.,.,chr22:10511359:TAA:T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274833,22,50808257,50808283,TGGGTTAGGGTTAGGGGTTAGGGGTTA,*,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,chr22:50808257:TGGGTTAGGGTTAGGGGTTAGGGGTTA:*
1274834,22,50808257,50808257,T,A,intergenic,RPL23AP82;NONE,dist=8620;dist=NONE,.,.,...,.,.,.,.,.,.,.,.,.,chr22:50808257:T:A
1274835,22,50808269,50808269,A,C,intergenic,RPL23AP82;NONE,dist=8632;dist=NONE,.,.,...,.,.,.,.,.,.,.,.,.,chr22:50808269:A:C
1274836,22,50808269,50808284,AGGGGTTAGGGGTTAG,*,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,chr22:50808269:AGGGGTTAGGGGTTAG:*


In [ ]:
af = anno.AF

In [ ]:
af = af.replace('.',0)

In [ ]:
af = af.astype(float)

In [ ]:
anno[af>0.01].Otherinfo1

0                                         chr22:10510356:T:A
6                                     chr22:10515040:T:TAAGA
7                                     chr22:10515040:TAAGA:T
12                             chr22:10515064:AAAGAAAGAAAG:A
17                                 chr22:10515067:GAAAGAAA:G
                                 ...                        
1274830                                  chr22:50808257:T:TC
1274832                                  chr22:50808257:T:TA
1274834                                   chr22:50808257:T:A
1274835                                   chr22:50808269:A:C
1274837    chr22:50808308:AGGGTTAGGGTTAGGGTTAGGCTTAGGGTTA...
Name: Otherinfo1, Length: 258728, dtype: object

In [ ]:
anno[af>0.01].Otherinfo1.to_csv('../data/runpmdata/variants_chr22_cutoff001.txt',header=False,index=False)

### Extract chr22 from vcf

plink --vcf /mnt/mfs/statgen/alzheimers-family/linkage_files/geno/full_sample/vcf/full_sample.vcf.gz --chr 22 --double-id --out chr22_cutoff001

plink --bfile chr22_cutoff001 --extract variants_chr22_cutoff001.txt --make-bed --out chr22_cutoff001_new

In [ ]:
import sys
import os.path
import glob
import pandas as pd
import numpy as np
import pickle
from SEQLinkage.linkage import *

In [ ]:
fam_vcf='../data/wg20220316/fam17_vcf.pickle'
fam_path='../data/new_trim_ped_famless17_no:xx.fam'
if os.path.isfile(fam_vcf):
    with open(fam_vcf, 'rb') as handle:
        fam17_vcf = pickle.load(handle)
fam17 = pd.read_csv(fam_path,delim_whitespace=True,header=None,names=['fid','iid','fathid','mothid','sex','ad'])
fam17.index = list(fam17.iid)
fam17.ad[fam17.ad==-9]=0
fam17_d = {}
cutoff=0.05
for i in fam17.fid.unique():
    fam17_d[i] = fam17[fam17.fid==i]

/tmp/2485848.1.high_mem.q/ipykernel_7047/2986720832.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fam17.ad[fam17.ad==-9]=0


In [ ]:
fam17

,fid,iid,fathid,mothid,sex,ad
1007_39,1007,1007_39,1007_40,1007_4,1,1
1007_99,1007,1007_99,1007_1,1007_2,2,2
1007_3,1007,1007_3,1007_1,1007_2,1,2
1007_5,1007,1007_5,1007_1,1007_2,2,2
1007_40,1007,1007_40,0,0,1,0
...,...,...,...,...,...,...
989_10,989,989_10,989_1,989_2,1,2
990_99,990,990_99,990_1,990_2,2,2
990_2,990,990_2,0,0,2,0
990_12,990,990_12,990_1,990_2,2,2


In [ ]:
from pandas_plink import read_plink

In [ ]:
chr22=read_plink('../data/runpmdata/chr22_cutoff001_new')

Mapping files: 100%|██████████| 3/3 [00:01<00:00,  1.70it/s]


In [ ]:
def bed2tped(genos,fams,file):
    '''genos=[bim,fam,geno]'''
    nvar=genos[0].shape[0]
    with open(file+'.ped', 'w') as f:
        for iid,j in fams.iterrows():
            genotypes=[str(t) for t in j]
            idx=genos[1].iid==iid
            if idx.sum()>0:
                for i in genos[2][:,list(idx)].compute():
                    if i[0]==0:
                        genotypes.extend(['1','1'])
                    elif i[0]==1:
                        genotypes.extend(['1','2'])
                    elif i[0]==2:
                        genotypes.extend(['2','2'])
                    else:
                        genotypes.extend(['0','0'])
            else:
                genotypes+=['0','0']*nvar
            f.write('\t'.join(genotypes))
            f.write('\n')
            
    genos[0].iloc[:,:4].to_csv(file+'.map',header=False,index=False,sep='\t')

### 1.The input of chr22 among all populations

In [ ]:
bed2tped(chr22,fam17,'../data/runpmdata/chr22_allfams_cutoff001')

In [ ]:
with open('../data/fam_pop.pickle', 'rb') as handle:
    fam_pop=pickle.load(handle)

### 2.The input of chr22 in amr

### 3.The input of chr22 in nfe